# Sistema de Recomendación de Cultivos
## Parte 2: Modelado y Evaluación (Fases 3-6 CRISP-DM)

**Machine Learning**

**Profesor:** Marco Terán  
**Fecha:** 2025/10/22

**Integrantes**

- Daniel Alejandro Garcia Zuluaica
- Edward Alejandro Rayo Cortés
- Elizabeth Toro Chalarca

___

## 📋 Tabla de Contenidos

1. **Resumen del EDA** (Referencia del Notebook 1)
2. **Fase 3: Preparación de Datos**
3. **Fase 4: Modelado**
4. **Fase 5: Evaluación**
5. **Fase 6: Despliegue**
6. **Conclusiones y Próximos Pasos**

---

## 📚 Resumen del EDA (Notebook 1)

Este notebook continúa el trabajo realizado en [1_crops_eda.ipynb](1_crops_eda.ipynb). A continuación, un resumen ejecutivo de los hallazgos clave:

### Calidad de Datos: ⭐⭐⭐⭐⭐ EXCELENTE

- **Completitud**: 100% (sin valores faltantes)
- **Balance**: Perfecto (100 muestras × 22 cultivos = 2,200 muestras)
- **Variables**: 7 numéricas (N, P, K, temperature, humidity, ph, rainfall)
- **Target**: 22 clases de cultivos

### Hallazgos Clave del EDA:

1. ✅ **Outliers válidos**: Representan requerimientos agronómicos específicos (conservar)
2. ✅ **Variables significativas**: Todas con F-statistic alto (p < 0.05)
3. ✅ **Separabilidad confirmada**: Silhouette Score t-SNE = 0.528 (buena separación)
4. ⚠️ **Distribuciones no normales**: Requiere transformación para modelos paramétricos
5. ⚠️ **Multicolinealidad P-K**: Correlación 0.736 (manejable con regularización)

### Decisiones para Modelado:

| Decisión | Justificación |
|----------|---------------|
| **Conservar outliers** | Agronómicamente válidos |
| **Usar 7 variables** | Todas discriminan significativamente |
| **RobustScaler** | Resistente a outliers |
| **PowerTransformer** | Para modelos paramétricos |
| **Modelos no-lineales** | PCA mostró que no hay separación lineal |

### Expectativas de Performance:

- **Objetivo mínimo**: 85% accuracy
- **Objetivo realista**: 90-93% accuracy
- **Objetivo óptimo**: ≥95% accuracy

### Modelos Prioritarios:

1. 🥇 **Random Forest**: Robusto, maneja outliers, captura interacciones
2. 🥇 **XGBoost**: State-of-art para multiclase
3. 🥈 **Logistic Regression**: Baseline interpretable
4. 🥈 **SVM (RBF)**: Bueno para no-linealidad

---

## 1. Configuración del Entorno

In [ ]:
pip install -r ../requirements.txt

In [7]:
# Configuración inicial
import sys
import warnings
warnings.filterwarnings('ignore')

# Verificar versión de Python
assert sys.version_info >= (3, 7), "Este notebook requiere Python 3.7 o superior"
print(f"✅ Python {sys.version_info.major}.{sys.version_info.minor} instalado correctamente")

✅ Python 3.12 instalado correctamente


In [8]:
# Importar librerías para modelado
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Scikit-learn para modelado
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, RobustScaler, PowerTransformer, LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, classification_report, ConfusionMatrixDisplay
)

# Modelos de clasificación
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

# XGBoost (si está instalado)
try:
    import xgboost as xgb
    print("✅ XGBoost disponible")
except ImportError:
    print("⚠️  XGBoost no instalado (opcional)")

# Configuración de visualización
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11
sns.set_palette("husl")

# Configuración de pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 4)

# Random state para reproducibilidad
RANDOM_STATE = 42

print("\n✅ Librerías importadas correctamente")

✅ XGBoost disponible

✅ Librerías importadas correctamente


In [9]:
# Importar funciones utilitarias
sys.path.append('../src')

# Recargar data_loader para obtener los datos
import importlib
import data_loader
importlib.reload(data_loader)

from data_loader import load_crop_data

---

## 2. Carga de Datos

Cargamos el dataset limpio (ya validado en el EDA):

In [10]:
# Cargar datos
crop_data = load_crop_data()

print("="*80)
print("RESUMEN DEL DATASET".center(80))
print("="*80)
print(f"Dimensiones: {crop_data.shape[0]} filas × {crop_data.shape[1]} columnas")
print(f"Cultivos únicos: {crop_data['label'].nunique()}")
print(f"\nPrimeras filas:")
display(crop_data.head())

print(f"\nDistribución de clases:")
print(crop_data['label'].value_counts().head(10))

📁 Cargando datos desde caché local: Crop_recommendation.csv
                              RESUMEN DEL DATASET                               
Dimensiones: 2200 filas × 8 columnas
Cultivos únicos: 22

Primeras filas:


,N,P,K,temperature,humidity,ph,rainfall,label
0,90,42,43,20.8797,82.0027,6.5030,202.9355,rice
1,85,58,41,21.7705,80.3196,7.0381,226.6555,rice
2,60,55,44,23.0045,82.3208,7.8402,263.9642,rice
3,74,35,40,26.4911,80.1584,6.9804,242.8640,rice
4,78,42,42,20.1302,81.6049,7.6285,262.7173,rice



Distribución de clases:
label
rice           100
maize          100
chickpea       100
kidneybeans    100
pigeonpeas     100
mothbeans      100
mungbean       100
blackgram      100
lentil         100
pomegranate    100
Name: count, dtype: int64


---

## 🔧 Fase 3: Preparación de Datos

Basándonos en las conclusiones del EDA, implementaremos:

1. **Separación de features y target**
2. **Encoding del target** (22 cultivos → 0-21)
3. **Split estratificado** (80% train, 20% test)
4. **Pipelines de preprocesamiento** (según tipo de modelo)
5. **Validación de la preparación**

### 3.1. Separación de Features y Target

In [ ]:
# Placeholder: Código de preparación de datos
print("⏳ Sección en desarrollo...")

---

## 🤖 Fase 4: Modelado

*Esta sección se desarrollará después de completar la Fase 3*

---

## 📊 Fase 5: Evaluación

*Esta sección se desarrollará después de completar la Fase 4*

---

## 🚀 Fase 6: Despliegue

*Esta sección se desarrollará después de completar la Fase 5*

---

## 📚 Referencias

Ver referencias completas en `1_crops_eda.ipynb`